In [14]:
from tax_inspector_competition import CompetitionSimulator, DayRoute, POI
from typing import List, Dict, Tuple
from csv_starting_points import *
import pandas

class Traveler: 
    def __init__(self, dataset_path: str):
        self.simulator = CompetitionSimulator(dataset_path)
        self.inspectors = []
        self.competition_results = {}
        self.fixed_starting_points = None
    def _high_value_strategy(self, starting_point: POI) -> List[POI]:
        """Strategia che privilegia POIs con valore alto"""
        jurisdiction_pois = self.simulator.optimizer.jurisdictions.get(starting_point.jurisdiction, [])
        available_pois = [poi for poi in jurisdiction_pois if poi.id != starting_point.id]
        
        # Ordina per valore decrescente
        available_pois.sort(key=lambda poi: poi.fee_value, reverse=True)
        
        # Prova ad aggiungere POIs in ordine di valore
        selected_pois = []
        for poi in available_pois:
            test_route = selected_pois + [poi]
            if (len(test_route) <= 8 and 
                self.simulator.optimizer.is_valid_route(starting_point, test_route)):
                selected_pois.append(poi)
        
        return selected_pois

In [4]:
t = Traveler("dataset_finale.csv")

In [27]:
df_dum = pd.read_csv('coordinate_per_giorno.csv')
df_dum.columns = ["day", "lat", "lon"]
df_dum.to_csv("st_points.csv")
starting_points = load_starting_points_from_coordinates_csv('st_points.csv')

if starting_points:
    print(f"\nDettagli punti di partenza:")
    for i, poi in enumerate(starting_points, 1):
        print(f"  Giorno {i}: ({poi.lat}, {poi.lon}) -> {poi.jurisdiction}")

Caricati 5 punti di partenza da 'st_points.csv'
Distribuzione per giurisdizione:
   J2: 1 punti
   J4: 1 punti
   J5: 2 punti
   J6: 1 punti

Dettagli punti di partenza:
  Giorno 1: (41.8907406, 12.4773952) -> J5
  Giorno 2: (41.9023265, 12.4988301) -> J6
  Giorno 3: (41.882561, 12.514131) -> J4
  Giorno 4: (41.8970256, 12.4744365) -> J5
  Giorno 5: (41.8461723, 12.4879358) -> J2


In [28]:
starting_points

[POI(id=9000, lat=41.8907406, lon=12.4773952, poi_type='starting_point', fee_value=0.0, jurisdiction='J5'),
 POI(id=9001, lat=41.9023265, lon=12.4988301, poi_type='starting_point', fee_value=0.0, jurisdiction='J6'),
 POI(id=9002, lat=41.882561, lon=12.514131, poi_type='starting_point', fee_value=0.0, jurisdiction='J4'),
 POI(id=9003, lat=41.8970256, lon=12.4744365, poi_type='starting_point', fee_value=0.0, jurisdiction='J5'),
 POI(id=9004, lat=41.8461723, lon=12.4879358, poi_type='starting_point', fee_value=0.0, jurisdiction='J2')]

In [31]:
t._high_value_strategy(starting_points[0])

[POI(id=5232181090, lat=41.89043045043945, lon=12.43865585327148, poi_type='swimming_pool', fee_value=240.3356905639248, jurisdiction='J5'),
 POI(id=5829710985, lat=41.90057373046875, lon=12.47997665405273, poi_type='sports', fee_value=229.9843252238471, jurisdiction='J5'),
 POI(id=6043589034, lat=41.9146728515625, lon=12.46701622009277, poi_type='sports', fee_value=229.9843252238471, jurisdiction='J5'),
 POI(id=10873539265, lat=41.90494537353516, lon=12.4788932800293, poi_type='sports', fee_value=229.9843252238471, jurisdiction='J5'),
 POI(id=12018507650, lat=41.90374374389648, lon=12.48329448699951, poi_type='sports', fee_value=229.9843252238471, jurisdiction='J5'),
 POI(id=2244954388, lat=41.89538955688477, lon=12.47938346862793, poi_type='nightclub', fee_value=204.6829716315392, jurisdiction='J5')]